# Elasticsearch-workshop 
From tutorial here: https://github.com/amboulouma/elasticsearch-workshop

In [5]:
!python -m pip install elasticsearch

In [1]:
from elasticsearch import Elasticsearch

In [24]:
# Elastic search configuation

es = Elasticsearch(HOST="http://localhost", PORT=9200)
es = Elasticsearch()

In [8]:
# Creating index

es.indices.create(index="first_index")

C:\Python39\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'first_index'}

In [9]:
# Looking if the index exists

es.indices.exists(index="first_index")

True

In [10]:
# Delete index

es.indices.delete(index="first_index")

{'acknowledged': True}

In [11]:
es.indices.exists(index="first_index")


False

In [12]:
# Inserting data

doc_1 = {"city": "Paris", "country": "France"}
doc_2 = {"city": "Vienna", "country": "Austria"}
doc_3 = {"city": "London", "country": "England"}

es.index(index="cities", doc_type="places", id=1, body=doc_1)

C:\Python39\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'cities',
 '_type': 'places',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [13]:
es.index(index="cities", doc_type="places", id=2, body=doc_2)


{'_index': 'cities',
 '_type': 'places',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [14]:
# Getting the data

res = es.get(index="cities", doc_type="places", id=1)

res

C:\Python39\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: [types removal] Specifying types in document get requests is deprecated, use the /{index}/_doc/{id} endpoint instead.
  warnings.warn(message, category=ElasticsearchWarning)


{'_index': 'cities',
 '_type': 'places',
 '_id': '1',
 '_version': 1,
 '_seq_no': 0,
 '_primary_term': 1,
 'found': True,
 '_source': {'city': 'Paris', 'country': 'France'}}

## Search query and matching documents


In [15]:
# Creating our data

doc_1 = {"sentence":"Hack COVID-19 is amazing!"}
doc_2 = {"sentence":"Hack-Quarantine is stunning!"}

es.index(index="english", doc_type="sentences", id=1, body=doc_1)
es.index(index="english", doc_type="sentences", id=2, body=doc_2)

{'_index': 'english',
 '_type': 'sentences',
 '_id': '2',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1,
 '_primary_term': 1}

In [16]:
# Creating our query

## Match query 

body = {
    "from":0,
    "size":0,
    "query": {
        "match": {
            "sentence":"Hack"
        }
    }
}

res = es.search(index="english", body=body)
res

{'took': 69,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [17]:
body = {
    "from":0,
    "size":2,
    "query": {
        "match": {
            "sentence":"Hack Quarantine"
        }
    }
}

res = es.search(index="english", body=body)
res

{'took': 63,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.9171577,
  'hits': [{'_index': 'english',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.9171577,
    '_source': {'sentence': 'Hack-Quarantine is stunning!'}},
   {'_index': 'english',
    '_type': 'sentences',
    '_id': '1',
    '_score': 0.17439456,
    '_source': {'sentence': 'Hack COVID-19 is amazing!'}}]}}

In [18]:
# match_phrase 

body = {
    "from":0,
    "size":1,
    "query": {
        "match_phrase": {
            "sentence":"Hack Quarantine"
        }
    }
}

res = es.search(index="english", body=body)
res

{'took': 20,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.91715777,
  'hits': [{'_index': 'english',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.91715777,
    '_source': {'sentence': 'Hack-Quarantine is stunning!'}}]}}

## Combining queries


In [20]:
# must, must_not and should
 
body = {
    "from":0,
    "size":2,
    "query": {
        "bool": {
            "must_not": {
                "match": {
                    "sentence":"COVID-19"
                }
            },
            "should": {
                "match": {
                    "sentence": "Hack"
                }
            }
        }
    }
}

res = es.search(index="english", body=body)
res

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.19100355,
  'hits': [{'_index': 'english',
    '_type': 'sentences',
    '_id': '2',
    '_score': 0.19100355,
    '_source': {'sentence': 'Hack-Quarantine is stunning!'}}]}}

## Regular Expressions Queries


In [21]:
# Updating our data

doc_1 = {"sentence":"Hack COVID-19 is amazing!"}
doc_2 = {"sentence":"Hack-Quarantine is stunning!"}
doc_3 = {"sentence":"Hack nCov is great!"}

es.index(index="english", doc_type="sentences", id=1, body=doc_1)

{'_index': 'english',
 '_type': 'sentences',
 '_id': '1',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [22]:
es.index(index="english", doc_type="sentences", id=2, body=doc_2)


{'_index': 'english',
 '_type': 'sentences',
 '_id': '2',
 '_version': 2,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 3,
 '_primary_term': 1}

In [23]:
# regexp query

body = {
    "from":0,
    "size":3,
    "query": {
        "regexp": {
            "sentence":".*"
        }
    }
}

res = es.search(index="english", body=body)
res

{'took': 524,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'english',
    '_type': 'sentences',
    '_id': '1',
    '_score': 1.0,
    '_source': {'sentence': 'Hack COVID-19 is amazing!'}},
   {'_index': 'english',
    '_type': 'sentences',
    '_id': '2',
    '_score': 1.0,
    '_source': {'sentence': 'Hack-Quarantine is stunning!'}}]}}

https://www.youtube.com/watch?v=ma3BC8aPBfE  
https://stackoverflow.com/questions/56833936/connection-error-while-using-the-python-elasticsearch  
https://www.elastic.co/guide/en/elastic-stack-get-started/current/get-started-elastic-stack.html  


---